# Computer Assignment #2 Game Report
# AI course - Fall 1400
<h2>By Kianoush Arshi<br>Std ID: 810198438</h2>


<h2>Part 1: Genes and Chromosomes</h2>
In this algorithm we have 3 different classes of data:<br>
<li>Genome, which is the smallest and mine is each cell of the board.<br>
<li>Chromosme, which is a cluster of genes together and mine is a 9 in 9 board.<br>
<li>Population, which is a list of chromosmes and my population size is 200.<br></li>

<h2>Genome</h2>
<p>Each cell is a genome, in the algorithms mutation function we swap 2 genes in the same chromosome and in the cross over we swap a group of genes between two chromosomes.<br></p>

<h2>Chromosome</h2>
<p>
    Each chromosome is a possible solution to our problem.<br>
    We can find out if a solution is found using a fitness function which is defined as follows:<br>
    <li>Get the count of duplicate numbers in each column and divide them by (9*9)</li>
    <li>Get the count of duplicate numbers in each 3by3 block and divide them by (9*9)</li>
    <li>Check if there is any duplicates:
        <li>If there isn't any: set fitness to 1 and return.
        <li>If there is: return the multitude of block_sum and col_sum as fitness.
    </li>    
</p>

<h2>Population</h2>
<p>
    We have a ppulation size of 200. Initial population is generated randomly with the only two constraints of not having duplicated any of the fixed values in a block, row or column and not inputted more than 9 times.<br>
    Each generation, we take 20% of the top(fittesst) chromosomes and keep them as elites in the next generation. Using these elites, we create children by randomly crossing over two of them and possibly mutating the child.<br>
    <h4>Cross Over:</h4>
    <p>
        A child is created using a cross over between two randomly selected chromosomes and then using two randomly generated cross over points, switched genes between these points in each row.
        The aforementioned approach for cross over had some problems, only the elites had a chance to be a parent. This might cause the algorithm to get stuck in local minima. Thus a new approach is introduced:<br>
        Have a competition between two randomly chosen chromosomes and give the weaker one a chance to be chosen reducing the chance for the population to go stale.
    </p>
    <h4>Muation:</h4>
    <p>
        In general, optimization solving algorithms converge to a local minimum.  To get out of this local minimum in a Genetic Algorithm, you can use mutations. The higher the mutation rate is, the more 'space' will be searched and the higher the chance that the global minimum is found. But if the mutation rate is too high, the algorithm won't converge anymore.<br>
        The cross overed child is then mutated with an adaptive mutation rate which chaanges each time we encounter a successful mutation and resets each time we encounter stale population.<br>
        Significant progress toward the fitness function's optimum, generally, can only happen in a narrow band of the mutation step size sigma. That narrow band is called evolution window.<br>
        There are three well-known methods to adapt the mutation step size σ in evolution strategies:<br>
            <li>(1/5-th) Success Rule</li>
            <li>Self-Adaptation (for example through log-normal mutations)</li>
            <li>Cumulative Step Size Adaptation (CSA)</li>
    </p>
    <h4>Handling Stale Population:</h4>
    <p>
        To detect a stale population, we have a counter which counts each time we see that the two best chromosomes have the same fitness. If this counter reaches 100, we reset the algorithm with new initial population and the base mutation rate.
    </p>
</p>

In [3]:
def read_file(filename : str) -> list() :
    file = open(filename, 'r+')
    init_board = []
    for line in file.readlines():
        init_board.append([int(i) for i in line.split()])
        
    return init_board

def print_chromosome(chromosome):
    if chromosome == None:
        return
    for i in range(len(chromosome)):
        for j in range(len(chromosome[0])):
            print(chromosome[i][j], end = " ")
        print()
    print()    


In [4]:
def is_row_duplicate(row, value, chromosome):
    for column in range(0, 9):
        if chromosome[row][column] == value:
            return True
    return False

def is_column_duplicate(column, value, chromosome):
    for row in range(0, 9):
        if chromosome[row][column] == value:
            return True
    return False

def is_block_duplicate(row, column, value, chromosome):
    i = 3 * (int(row / 3))
    j = 3 * (int(column / 3))

    if ((chromosome[i][j] == value)
        or (chromosome[i][j + 1] == value)
        or (chromosome[i][j + 2] == value)
        or (chromosome[i + 1][j] == value)
        or (chromosome[i + 1][j + 1] == value)
        or (chromosome[i + 1][j + 2] == value)
        or (chromosome[i + 2][j] == value)
        or (chromosome[i + 2][j + 1] == value)
        or (chromosome[i + 2][j + 2] == value)):
        return True
    else:
        return False

def make_index(v):
    if v <= 2:
        return 0
    elif v <= 5:
        return 3
    else:
        return 6
    
def find_unused(parent_row, remaining):
    for i in range(0, len(parent_row)):
        if (parent_row[i] in remaining):
            return i

def find_value(parent_row, value):
    for i in range(0, len(parent_row)):
        if (parent_row[i] == value):
            return i

In [5]:
def compete(chromosome1, chromosome2):

        # Find the fittest and the weakest.
        if (chromosome1[0] > chromosome2[0]):
            fittest = chromosome1[1]
            weakest = chromosome2[1]
        else:
            fittest = chromosome2[1]
            weakest = chromosome1[1]

        selection_rate = 0.80
        r = random.uniform(0, 1.1)
        while (r > 1):  # Outside [0, 1] boundary. Choose another.
            r = random.uniform(0, 1.1)
        if (r < selection_rate):
            return fittest
        else:
            return weakest

In [10]:
import random
from copy import deepcopy
from math import floor
import numpy as np

random.seed()

class SudokuSolver():

    def __init__(self, init_board):
        self.fixed_values = init_board
        self.solved = 0
        self.pop_size = 200  # population size
        self.elite_percentage = 10 # keep x% best of parents in the new population
        self.Ng = 10000  # Number of generations.
        self.m_count = 0  # Number of mutations.
        self.ans = []
        
        # Cross over probability
        self.pc = 0.7
        # Mutation parameters.
        self.phi = 0
        self.sigma = 1
        self.pm = 0.06

        # Create an initial population.
        self.chromosomes = self.generate_initial_population()
        return
    
    def generate_initial_population(self):
        self.stale = 0
        init_pop = []
        for t in range(self.pop_size):
            chromosome = []
            
            used_nums = [0]*9
            chromosome = deepcopy(self.fixed_values)
            available_values = list(range(1,10))

            for row in range(0, 9):
                for column in range(0, 9):
                    if self.fixed_values[row][column] != 0:
                        used_nums[self.fixed_values[row][column]-1] += 1

            # Determine the legal values that each square can take.
            for row in range(0, 9):
                for column in range(0, 9):
                    if self.fixed_values[row][column] != 0: # if value is already given
                        continue
                    while(True):
                        value = available_values[random.randint(0, len(available_values)-1)]
                        if (self.fixed_values[row][column] == 0):
                            used_nums[value-1] += 1
                            if used_nums[value-1] == 9:
                                available_values.remove(value)
                            chromosome[row][column] = value
                            break

            init_pop.append(chromosome)
            
        print_chromosome(init_pop[0])
        print("Initial fitness:", self.get_fitness(init_pop[0]))
        return init_pop
    
    def cross_over(self, chromosome1, chromosome2):
        # Make a copy of the parent genes.
        child1 = deepcopy(chromosome1)
        child2 = deepcopy(chromosome2)

        # Pick a crossover point. Crossover must have at least 1 row (and at most 8) rows.
        crossover_point1 = random.randint(0, 8)
        crossover_point2 = random.randint(1, 9)
        while (crossover_point1 == crossover_point2):
            crossover_point1 = random.randint(0, 8)
            crossover_point2 = random.randint(1, 9)

        if (crossover_point1 > crossover_point2):
            temp = crossover_point1
            crossover_point1 = crossover_point2
            crossover_point2 = temp
            
        for i in range(crossover_point1, crossover_point2):
            child1[i], child2[i] = self.crossover_rows(child1[i], child2[i])

        return child1, child2
                    
    def crossover_rows(self, row1, row2):
        child_row1 = np.zeros(9)
        child_row2 = np.zeros(9)

        remaining = [i for i in range(1, 10)]
        cycle = 0

        while ((0 in child_row1) and (0 in child_row2)):
            if (cycle % 2 == 0):  # Even cycles.
                # Assign next unused value.
                index = find_unused(row1, remaining)
                start = row1[index]
                remaining.remove(row1[index])
                child_row1[index] = row1[index]
                child_row2[index] = row2[index]
                next = row2[index]

                while (next != start):  # While cycle not done...
                    index = find_value(row1, next)
                    child_row1[index] = row1[index]
                    remaining.remove(row1[index])
                    child_row2[index] = row2[index]
                    next = row2[index]

                cycle += 1

            else:  # Odd cycle - flip values.
                index = find_unused(row1, remaining)
                start = row1[index]
                remaining.remove(row1[index])
                child_row1[index] = row2[index]
                child_row2[index] = row1[index]
                next = row2[index]

                while (next != start):  # While cycle not done...
                    index = find_value(row1, next)
                    child_row1[index] = row2[index]
                    remaining.remove(row1[index])
                    child_row2[index] = row1[index]
                    next = row2[index]

                cycle += 1

        return child_row1, child_row2

    def mutate(self, chromosome):
        prob = random.random()
        if (prob < self.pm):  # Mutate.
            row1, col1 = (random.randint(0, 8), random.randint(0, 8))
            row2, col2 = (random.randint(0, 8), random.randint(0, 8))

            while (row1, col1) == (row2, col2):
                row1, col1 = (random.randint(0, 8), random.randint(0, 8))
                row2, col2 = (random.randint(0, 8), random.randint(0, 8))
            row2 = row1

            # Check if the two places are not fixed
            if self.fixed_values[row1][col1] == 0 and self.fixed_values[row2][col2] == 0:
                temp_chromosome = deepcopy(chromosome)
                temp = temp_chromosome[row2][col2]
                temp_chromosome[row2][col2] = temp_chromosome[row1][col1]
                temp_chromosome[row1][col1] = temp
                # and we are not lowering fitness.
                if self.get_fitness(temp_chromosome) > self.get_fitness(chromosome):
                    self.m_count += 1
                    self.phi += 1
                    chromosome = deepcopy(temp_chromosome)

        return chromosome
        
        
    def get_fitness(self, chromosome):
        column_count = [0]*9
        block_count = [0]*9
        column_sum = 0
        block_sum = 0
        
        # For each column....
        for j in range(0, 9):
            for i in range(0, 9):
                column_count[chromosome[i][j] - 1] += 1

            for k in range(9):
                if column_count[k] == 1:
                    column_sum += (1/9)/9
            column_count = [0]*9

        # For each block...
        for i in range(0, 9, 3):
            for j in range(0, 9, 3):
                block_count[chromosome[i][j] - 1] += 1
                block_count[chromosome[i][j + 1] - 1] += 1
                block_count[chromosome[i][j + 2] - 1] += 1

                block_count[chromosome[i + 1][j] - 1] += 1
                block_count[chromosome[i + 1][j + 1] - 1] += 1
                block_count[chromosome[i + 1][j + 2] - 1] += 1

                block_count[chromosome[i + 2][j] - 1] += 1
                block_count[chromosome[i + 2][j + 1] - 1] += 1
                block_count[chromosome[i + 2][j + 2] - 1] += 1

                for k in range(9):
                    if block_count[k] == 1:
                        block_sum += (1/9)/9
                block_count = [0]*9

        # Calculate overall fitness.
        if column_sum == 1 and block_sum == 1:
            fitness = 1
        else:
            fitness = column_sum * block_sum

        return fitness

    
    def generate_new_population(self, generation):
        sorted_chromosomes = [(self.get_fitness(chromosome), chromosome) for chromosome in self.chromosomes]
        sorted_chromosomes.sort(reverse=True)
        
        # Check for stale population.
        if (sorted_chromosomes[0] != sorted_chromosomes[1]):
            self.stale = 0
        else:
            self.stale += 1

        # Re-seed the population if 100 generations have passed
        # with the fittest two candidates always having the same fitness.
        if (self.stale >= 100):
            print("The population has gone stale. Re-seeding...")
            self.sigma = 1
            self.phi = 0
            self.pm = 0.06
            return self.generate_initial_population()

        elites = sorted_chromosomes[:floor((self.elite_percentage / 100) * (self.pop_size))]
        print("Generation {}. Best fitness: {}. Second best: {}".format(generation, elites[0][0], elites[1][0]))
        if (sorted_chromosomes[0][0] == 1): #no intersections in the best chromosome
            self.solved = 1
            self.ans = sorted_chromosomes[0][1]
            return sorted_chromosomes[0]

        new_pop = [chromosome[1] for chromosome in elites] # add elites

        # Create the rest of the candidates.
        while (len(new_pop) < self.pop_size):
            # Select parents from population via a tournament.
            potential_parents = random.sample(sorted_chromosomes, 4)
            parent1 = compete(potential_parents[0], potential_parents[1])
            parent2 = compete(potential_parents[2], potential_parents[3])

            child1, child2 = self.cross_over(parent1, parent2)

            mutated_child1 = self.mutate(child1)
            mutated_child2 = self.mutate(child2)
            new_pop.append(mutated_child1)
            new_pop.append(mutated_child2)

        return new_pop

    def solve(self):        
        for generation in range(0, self.Ng):
            # Create the next population.
            self.chromosomes = self.generate_new_population(generation)
            # Calculate new adaptive mutation rate (based on Rechenberg's 1/5 success rule).
            # This is to stop too much mutation as the fitness progresses towards unity.
            if (self.m_count == 0):
                self.phi = 0  # Avoid divide by zero.
            else:
                self.phi = self.phi / self.m_count

            if (self.phi > 0.2):
                self.sigma = self.sigma / 0.998
            elif (self.phi < 0.2):
                self.sigma = self.sigma * 0.998
                
            self.pm = abs(np.random.normal(loc=0.0, scale=self.sigma, size=None))
                
            if self.solved:
                return self.ans

        print("No solution found.")
        return None
    
    

from time import time

if __name__ == '__main__':

    sudoku_solver = SudokuSolver(read_file("Test1.txt"))

    start = time()

    print_chromosome(sudoku_solver.solve())

    end = time()

    print("Time taken:", (end-start))

8 7 6 6 2 7 1 3 7 
2 5 8 6 4 2 6 6 8 
4 5 3 3 3 4 8 8 6 
7 1 4 8 8 2 6 3 3 
3 3 7 3 2 2 2 9 1 
1 9 5 5 4 9 4 6 7 
9 4 1 2 5 8 7 5 9 
2 7 4 4 9 6 9 1 5 
5 7 9 1 1 5 1 8 9 

Initial fitness: 0.2220698064319465
Generation 0. Best fitness: 0.30864197530864224. Second best: 0.30727023319615937
[1, 2, 3, 4, 5, 6, 7, 8, 9]
7
0


TypeError: list indices must be integers or slices, not NoneType

In [15]:
class Sudoku(object):
    """ Solves a given Sudoku puzzle using a genetic algorithm. """

    def __init__(self):
        self.given = None
        return

    def load(self, p):
        #values = np.array(list(p.replace(".","0"))).reshape((Nd, Nd)).astype(int)
        self.given = Fixed(p)
        return

    def solve(self):

        Nc = 1000  # Number of candidates (i.e. population size).
        Ne = int(0.05 * Nc)  # Number of elites.
        Ng = 10000  # Number of generations.
        Nm = 0  # Number of mutations.

        # Mutation parameters.
        phi = 0
        sigma = 1
        mutation_rate = 0.06

        # Check given one first
        if self.given.no_duplicates() == False:
            return (-1, 1)

        # Create an initial population.
        self.population = Population()
        print("create an initial population.")
        if self.population.seed(Nc, self.given) ==  1:
            pass
        else:
            return (-1, 1)

        # For up to 10000 generations...
        stale = 0
        for generation in range(0, Ng):

            # Check for a solution.
            best_fitness = 0.0
            #best_fitness_population_values = self.population.candidates[0].values
            for c in range(0, Nc):
                fitness = self.population.candidates[c].fitness
                if (fitness == 1):
                    print("Solution found at generation %d!" % generation)
                    return (generation, self.population.candidates[c])

                # Find the best fitness and corresponding chromosome
                if (fitness > best_fitness):
                    best_fitness = fitness
                    #best_fitness_population_values = self.population.candidates[c].values

            print("Generation:", generation, " Best fitness:", best_fitness)
            #print(best_fitness_population_values)

            # Create the next population.
            next_population = []

            # Select elites (the fittest candidates) and preserve them for the next generation.
            self.population.sort()
            elites = []
            for e in range(0, Ne):
                elite = Candidate()
                elite.values = np.copy(self.population.candidates[e].values)
                elites.append(elite)

            # Create the rest of the candidates.
            for count in range(Ne, Nc, 2):
                # Select parents from population via a tournament.
                t = Tournament()
                parent1 = t.compete(self.population.candidates)
                parent2 = t.compete(self.population.candidates)

                ## Cross-over.
                cc = CycleCrossover()
                child1, child2 = cc.crossover(parent1, parent2, crossover_rate=1.0)

                # Mutate child1.
                child1.update_fitness()
                old_fitness = child1.fitness
                success = child1.mutate(mutation_rate, self.given)
                child1.update_fitness()
                if (success):
                    Nm += 1
                    if (child1.fitness > old_fitness):  # Used to calculate the relative success rate of mutations.
                        phi = phi + 1

                # Mutate child2.
                child2.update_fitness()
                old_fitness = child2.fitness
                success = child2.mutate(mutation_rate, self.given)
                child2.update_fitness()
                if (success):
                    Nm += 1
                    if (child2.fitness > old_fitness):  # Used to calculate the relative success rate of mutations.
                        phi = phi + 1

                # Add children to new population.
                next_population.append(child1)
                next_population.append(child2)

            # Append elites onto the end of the population. These will not have been affected by crossover or mutation.
            for e in range(0, Ne):
                next_population.append(elites[e])

            # Select next generation.
            self.population.candidates = next_population
            self.population.update_fitness()

            # Calculate new adaptive mutation rate (based on Rechenberg's 1/5 success rule).
            # This is to stop too much mutation as the fitness progresses towards unity.
            if (Nm == 0):
                phi = 0  # Avoid divide by zero.
            else:
                phi = phi / Nm

            if (phi > 0.2):
                sigma = sigma / 0.998
            elif (phi < 0.2):
                sigma = sigma * 0.998

            mutation_rate = abs(np.random.normal(loc=0.0, scale=sigma, size=None))

            # Check for stale population.
            self.population.sort()
            if (self.population.candidates[0].fitness != self.population.candidates[1].fitness):
                stale = 0
            else:
                stale += 1

            # Re-seed the population if 100 generations have passed
            # with the fittest two candidates always having the same fitness.
            if (stale >= 100):
                print("The population has gone stale. Re-seeding...")
                self.population.seed(Nc, self.given)
                stale = 0
                sigma = 1
                phi = 0
                mutation_rate = 0.06

        print("No solution found.")
        return (-2, 1)

In [18]:
 s = Sudoku()
s.load(read_file("Test1.txt"))
start_time = time()
generation, solution = s.solve()
if (solution):
    if generation == -1:
        print("Invalid inputs")
        str_print = "Invalid input, please try to generate new game"
    elif generation == -2:
        print("No solution found")
        str_print = "No solution found, please try again"
    else:
        print(solution.values)
        time_elapsed = '{0:6.2f}'.format(time()-start_time)
        str_print = "Solution found at generation: " + str(generation) + \
                "\n" + "Time elapsed: " + str(time_elapsed) + "s"

create an initial population.
Generation: 0  Best fitness: 0.7683279987806766
Generation: 1  Best fitness: 0.7887517146776437
Generation: 2  Best fitness: 0.7887517146776437
Generation: 3  Best fitness: 0.8097850937357142
Generation: 4  Best fitness: 0.8271604938271638
Generation: 5  Best fitness: 0.8332571254381987
Generation: 6  Best fitness: 0.8116140832190247
Generation: 7  Best fitness: 0.8024691358024723
Generation: 8  Best fitness: 0.8567291571406833
Generation: 9  Best fitness: 0.8765432098765469
Generation: 10  Best fitness: 0.8802011888431678
Generation: 11  Best fitness: 0.8765432098765469
Generation: 12  Best fitness: 0.9012345679012383
Generation: 13  Best fitness: 0.9036732205456524
Generation: 14  Best fitness: 0.9012345679012383
Generation: 15  Best fitness: 0.9259259259259299
Generation: 16  Best fitness: 0.9259259259259299
Generation: 17  Best fitness: 0.9259259259259299
Generation: 18  Best fitness: 0.9012345679012383
Generation: 19  Best fitness: 0.9036732205456524
